In [1]:
print("test")

test


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Load data
def data_load(data):
    loaded = DirectoryLoader(
        data, 
        glob="*.pdf",
        loader_cls=PyPDFLoader
        )
    return loaded.load()

In [21]:
pdf_data = data_load('..\\Data\\')

In [4]:
#pdf_data

In [22]:
def text_split(pdf_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
        )
    text_chunk = text_splitter.split_documents(pdf_data)
    return text_chunk

In [23]:
text_chunk = text_split(pdf_data=pdf_data)

In [24]:
len(text_chunk)

7020

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
def huggingface_download():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
embeddings = huggingface_download()

C:\Users\HP\AppData\Local\Temp\ipykernel_22564\1870086992.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [28]:
len(embeddings.embed_query("Hi, My name is Segun"))

384

In [8]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [31]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "med-bot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
    
)

In [32]:
from langchain_pinecone import PineconeVectorStore

med_db = PineconeVectorStore.from_documents(
    documents=text_chunk,
    index_name=index_name,
    embedding=embeddings
)

In [10]:
from langchain_pinecone import PineconeVectorStore
index_name = "med-bot"
med_db = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [21]:
retriver = med_db.as_retriever()#search_type="similarity", search_kwargs={"k":3})

In [22]:
retriver.invoke("What are precautions")

[Document(id='1e531897-63f1-493d-a2e2-0670e8d823a8', metadata={'page': 260.0, 'source': 'Data\\Medical_book.pdf'}, page_content='fessional should also be sure to use standard precautions,including gowns and gloves when appropriate. Patientsshould be alerted to avoid risks of viral contamination,\nand live virus immunizations are contraindicated untilthe patient has fully recovered from the effects ofchemotherapy. Similarly, the patient should avoid contactwith other people who have recently had live virusimmunizations.\nOther precautions which should be emphasized are'),
 Document(id='a9b984e2-f2b4-436a-86a0-87dd32a4ce4f', metadata={'page': 516.0, 'source': 'Data\\Medical_book.pdf'}, page_content='CATS. Prevention involves warning children to stay\naway from strange cats and to avoid rough play and otherbehavior that can anger cats and cause them to bite.\nArthropods\nSPIDERS. Common-sense precautions include clear-\ning webs out of garages, outhouses, and other placesfavored by venomo

In [12]:
from langchain_openai import OpenAI
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = OpenAI(temperature=0.4, max_tokens=500)


In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for a question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. if you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the answer"
    "concise."
    "\n\n\n" 
    "{context}"

)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [14]:
q_a_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, q_a_chain)

In [15]:
resonse = rag_chain.invoke({"input":"What is Acne"})

In [17]:
resonse["answer"].replace('?\n\n','')

'Acne is a common skin disease that causes pimples on the face, chest, and back. It is caused by clogged pores due to oil, dead skin cells, and bacteria. Acne vulgaris is the medical term for common acne and can be treated with various topical medications. In severe cases, isotretinoin may be prescribed.'

In [79]:
resonse

{'input': 'What is Acne',
 'context': [Document(id='aab99082-e6df-42e1-afac-b2e4c3e58dc0', metadata={'page': 37.0, 'source': '..\\Data\\Medical_book.pdf'}, page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
  Document(id='2aaaa80d-d2bb-4966-88eb-efd6dde5e05a', metadata={'page': 38.0, 'source': '..\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')],
 'answer': '?\n\nSystem: Acne is a

In [26]:
system_prompt = """Answer the questions based only on the following context:
            {context}
            Question: {question}
            """


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


retriver = med_db.as_retriever()#search_type="similarity", search_kwargs={"k":2})
llm = OpenAI(temperature=0, max_tokens=500)


prompt = ChatPromptTemplate.from_template(system_prompt)
    #[
     #   ("system", system_prompt),
      #  ("human", "{input}"),
    #]
#)

chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser
    
    )

In [29]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n" 
    "{context}"
    "\n\n"
    "Human: {question}"
    "\n\n"
    "Assistant: "
)

prompt = ChatPromptTemplate.from_template(system_prompt)

chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Usage
question = "What is acne?"
response = chain.invoke(question)

# Remove the system prefix if it exists
cleaned_response = response.replace('?\n\nSystem: ', '')

cleaned_response

'Acne is a common skin disease that causes pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It is also known as acne vulgaris and can be treated with various medications.'

In [30]:
response

'Acne is a common skin disease that causes pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It is also known as acne vulgaris and can be treated with various medications.'